In [1]:
import torch, copy, time, itertools, random

import pandas as pd
import numpy as np

import torch
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from load_adult import *
from utils import *
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.sampler import Sampler

from functools import partial

In [2]:
def loss_func(option, logits, targets, distance, sensitive, mean_sensitive, larg = 1):
    acc_loss = F.cross_entropy(logits, targets, reduction = 'sum')
    fair_loss = torch.mul(sensitive - sensitive.type(torch.FloatTensor).mean(), distance.T[0])
    fair_loss = torch.mean(torch.mul(fair_loss, fair_loss)) # modified mean to sum
    if option == 'Zafar':
        return acc_loss + larg*fair_loss, acc_loss, larg*fair_loss
    else:
        return acc_loss, acc_loss, larg*fair_loss

In [3]:
class FairBatch(Sampler):
    """FairBatch (Sampler in DataLoader).
    
    This class is for implementing batch selection of FairBatch.
        
    """
    def __init__(self, train_dataset, lbd, client_idx, batch_size, replacement = False, seed = 0):
        """Initializes FairBatch."""
        
        np.random.seed(seed)
        random.seed(seed)

        self.batch_size = batch_size
        self.N = train_dataset.y.shape[0]
        self.batch_num = int(self.N / self.batch_size)
        self.lbd = lbd
        
        self.yz_index = {}
        
        for y, z in itertools.product([0,1], [0,1]):
                self.yz_index[(y,z)] = np.where((train_dataset.y == y) & (train_dataset.sen == z))[0]

    def select_batch_replacement(self, batch_size, full_index, batch_num, replacement = False):
        """Selects a certain number of batches based on the given batch size.
        
        Args: 
            batch_size: An integer for the data size in a batch.
            full_index: An array containing the candidate data indices.
            batch_num: An integer indicating the number of batches.
            replacement: A boolean indicating whether a batch consists of data with or without replacement.
        
        Returns:
            Indices that indicate the data.
            
        """
        
        select_index = []
        
        if replacement == True:
            for _ in range(batch_num):
                select_index.append(np.random.choice(full_index, batch_size, replace = False))
        else:
            tmp_index = full_index.copy()
            random.shuffle(tmp_index)
            
            start_idx = 0
            for i in range(batch_num):
                if start_idx + batch_size > len(full_index):
                    select_index.append(np.concatenate((tmp_index[start_idx:], tmp_index[ : batch_size - (len(full_index)-start_idx)])))
                    start_idx = len(full_index)-start_idx
                else:
                    select_index.append(tmp_index[start_idx:start_idx + batch_size])
                    start_idx += batch_size
        return select_index

    
    def __iter__(self):
        """Iters the full process of FairBatch for serving the batches to training.
        
        Returns:
            Indices that indicate the data in each batch.
            
        """

        # Get the indices for each class
        sort_index_y_1_z_1 = self.select_batch_replacement(int(self.lbd[(1,1)] * self.batch_size), self.yz_index[(1,1)], self.batch_num)
        sort_index_y_0_z_1 = self.select_batch_replacement(int(self.lbd[(0,1)] * self.batch_size), self.yz_index[(0,1)], self.batch_num)
        sort_index_y_1_z_0 = self.select_batch_replacement(int(self.lbd[(1,0)] * self.batch_size), self.yz_index[(1,0)], self.batch_num)
        sort_index_y_0_z_0 = self.select_batch_replacement(int(self.lbd[(0,0)] * self.batch_size), self.yz_index[(0,0)], self.batch_num)


        for i in range(self.batch_num):
            key_in_fairbatch = sort_index_y_0_z_0[i].copy()
            key_in_fairbatch = np.hstack((key_in_fairbatch, sort_index_y_1_z_0[i].copy()))
            key_in_fairbatch = np.hstack((key_in_fairbatch, sort_index_y_0_z_1[i].copy()))
            key_in_fairbatch = np.hstack((key_in_fairbatch, sort_index_y_1_z_1[i].copy()))

            random.shuffle(key_in_fairbatch)
            for idx in key_in_fairbatch:
                yield idx
            # yield key_in_fairbatch
                               

    def __len__(self):
        """Returns the length of data."""
        
        return self.N

In [4]:
class ClientUpdate(object):
    def __init__(self, dataset, idxs, batch_size, option, penalty = 0, lbd = None):
        self.trainloader, self.validloader = self.train_val(dataset, list(idxs), batch_size, option, lbd)
        self.dataset = dataset
        self.option = option
        self.penalty = penalty
            
    def train_val(self, dataset, idxs, batch_size, option, lbd):
        """
        Returns train, validation for a given local training dataset
        and user indexes.
        """
        
        # split indexes for train, validation (90, 10)
        idxs_train = idxs[:int(0.9*len(idxs))]
        idxs_val = idxs[int(0.9*len(idxs)):len(idxs)]
        
        if option == "FairBatch": 
            # FairBatch(self, train_dataset, lbd, client_idx, batch_size, replacement = False, seed = 0)
            sampler = FairBatch(DatasetSplit(dataset, idxs_train), lbd, idxs,
                                 batch_size = batch_size, replacement = False, seed = 0)
            trainloader = DataLoader(DatasetSplit(dataset, idxs_train), sampler = sampler,
                                     batch_size=batch_size, num_workers = 0)
                        
        else:
            trainloader = DataLoader(DatasetSplit(dataset, idxs_train),
                                     batch_size=batch_size, shuffle=True)

        validloader = DataLoader(DatasetSplit(dataset, idxs_val),
                                     batch_size=int(len(idxs_val)/10), shuffle=False)
        return trainloader, validloader

    def update_weights(self, model, global_round, learning_rate, local_epochs, optimizer):
        # Set mode to train model
        model.train()
        epoch_loss = []

        # Set optimizer for the local updates
        if optimizer == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                                        ) # momentum=0.5
        elif optimizer == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                                         weight_decay=1e-4)

        for i in range(local_epochs):
            batch_loss = []
            for batch_idx, (features, labels, sensitive) in enumerate(self.trainloader):
                features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
                # we need to set the gradients to zero before starting to do backpropragation 
                # because PyTorch accumulates the gradients on subsequent backward passes. 
                # This is convenient while training RNNs

                log_probs, logits = model(features)
                loss, _, _ = loss_func(self.option,
                    logits, labels, logits, sensitive, mean_sensitive, self.penalty)
                    
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if batch_idx % 50 == 0:
                    print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tBatch Loss: {:.6f}'.format(
                        global_round, i, batch_idx * len(features),
                        len(self.trainloader.dataset),
                        100. * batch_idx / len(self.trainloader), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        # weight, loss
        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

    def inference(self, model, option):
        """ 
        Returns the inference accuracy, 
                                loss, 
                                N(sensitive group, pos), 
                                N(non-sensitive group, pos), 
                                N(sensitive group),
                                N(non-sensitive group),
                                acc_loss,
                                fair_loss
        """

        model.eval()
        loss, total, correct, fair_loss, acc_loss, num_batch = 0.0, 0.0, 0.0, 0.0, 0.0, 0
        n_yz = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
        loss_yz = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
        
        dataset = self.validloader if option != "FairBatch" else self.dataset
        for batch_idx, (features, labels, sensitive) in enumerate(self.validloader):
            features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
            sensitive = sensitive.to(DEVICE)
            
            # Inference
            outputs, logits = model(features)

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            bool_correct = torch.eq(pred_labels, labels)
            correct += torch.sum(bool_correct).item()
            total += len(labels)
            num_batch += 1
            
            group_boolean_idx = {}
            
            
            for yz in n_yz:
                group_boolean_idx[yz] = (pred_labels == yz[0]) & (sensitive == yz[1])
                n_yz[yz] += torch.sum(group_boolean_idx[yz]).item()            
                
                if self.option == "FairBatch":
                # the objective function have no lagrangian term
                    loss_yz_,_,_ = loss_func("unconstrained", outputs[group_boolean_idx[yz]], 
                                                    labels[group_boolean_idx[yz]], 
                                         logits[group_boolean_idx[yz]], sensitive[group_boolean_idx[yz]], 
                                         mean_sensitive, self.penalty)
                    loss_yz[yz] += loss_yz_
            
            batch_loss, batch_acc_loss, batch_fair_loss = loss_func(self.option, outputs, 
                                                        labels, logits, sensitive, mean_sensitive, self.penalty)
            loss, acc_loss, fair_loss = (loss + batch_loss.item(), 
                                         acc_loss + batch_acc_loss.item(), 
                                         fair_loss + batch_fair_loss.item())
        accuracy = correct/total
        if option == "FairBatch":
            return accuracy, loss, n_yz, acc_loss / num_batch, fair_loss / num_batch, loss_yz
        else:
            return accuracy, loss, n_yz, acc_loss / num_batch, fair_loss / num_batch, None

# TODO: need to update the criterion to loss_func
def test_inference(model, test_dataset, batch_size):
    """ Returns the test accuracy and loss.
    """

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0
    n_yz = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
    
    criterion = nn.NLLLoss().to(DEVICE)
    testloader = DataLoader(test_dataset, batch_size=batch_size,
                            shuffle=False)

    for batch_idx, (features, labels, sensitive) in enumerate(testloader):
        features = features.to(DEVICE)
        labels =  labels.to(DEVICE).type(torch.LongTensor)
        # Inference
        outputs, logits = model(features)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        bool_correct = torch.eq(pred_labels, labels)
        correct += torch.sum(bool_correct).item()
        total += len(labels)
        
        for yz in n_yz:
            n_yz[yz] += torch.sum((pred_labels == yz[0]) & (sensitive == yz[1])).item()  
            
#         # classified negative, nonsensitive
#         n_yz[(0,0)] += torch.sum(torch.logical_and(torch.logical_not(pred_labels), torch.logical_not(sensitive))).item()
#         # classified negative, sensitive
#         n_yz[(0,1)] += torch.sum(torch.logical_and(torch.logical_not(pred_labels), sensitive)).item()
#         # classified positive, nonsensitive
#         n_yz[(1,0)] += torch.sum(torch.logical_and(pred_labels, torch.logical_not(sensitive))).item()
#         # classified positive, sensitive
#         n_yz[(1,1)] += torch.sum(torch.logical_and(pred_labels, sensitive)).item()

    accuracy = correct/total
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    return accuracy, loss, RD(n_yz)

In [5]:
def train(model, option = "unconstrained", batch_size = 128, num_clients = 2,
          num_rounds = 5, learning_rate = 0.01, optimizer = 'adam', local_epochs= 5, 
          num_workers = 4, print_every = 1,
         penalty = 1, alpha = 0.005, seed = 123):
    """
    Server execution.
    """
    
    np.random.seed(seed)
    random.seed(seed)
        
    # Training
    train_loss, train_accuracy = [], []
    val_acc_list, net_list = [], []
    cv_loss, cv_acc = [], []
    val_loss_pre, counter = 0, 0
    start_time = time.time()
    weights = model.state_dict()
    
    test_loader = DataLoader(dataset = test_dataset,
                            batch_size = batch_size,
                            num_workers = num_workers)
    
    train_loader = DataLoader(dataset = train_dataset,
                        batch_size = batch_size,
                        num_workers = num_workers)

    def average_weights(w):
        """
        Returns the average of the weights.
        """
        w_avg = copy.deepcopy(w[0])
        for key in w_avg.keys():
            for i in range(1, len(w)):
                w_avg[key] += w[i][key]
            w_avg[key] = torch.div(w_avg[key], len(w))
        return w_avg

    # the number of samples whose label is y and sensitive attribute is z
    m_yz = {(0,0): ((train_dataset.y == 0) & (train_dataset.sen == 0)).sum(),
           (1,0): ((train_dataset.y == 1) & (train_dataset.sen == 0)).sum(),
           (0,1): ((train_dataset.y == 0) & (train_dataset.sen == 1)).sum(),
           (1,1): ((train_dataset.y == 1) & (train_dataset.sen == 1)).sum()}
    
    lbd = {
        (0,0): m_yz[(0,0)]/train_dataset.y.shape[0], 
        (0,1): m_yz[(0,1)]/train_dataset.y.shape[0],
        (1,0): m_yz[(0,1)]/train_dataset.y.shape[0],
        (1,1): m_yz[(1,1)]/train_dataset.y.shape[0],
          }
    
    for round_ in tqdm(range(num_rounds)):
        local_weights, local_losses = [], []
        print(f'\n | Global Training Round : {round_+1} |\n')

        model.train()
        m = 2 # the number of clients to be chosen in each round_
        idxs_users = np.random.choice(range(num_clients), m, replace=False)

        for idx in idxs_users:
            local_model = ClientUpdate(dataset=train_dataset,
                                        idxs=clients_idx[idx], batch_size = batch_size, 
                                       option = option, penalty = penalty, lbd = lbd)
            w, loss = local_model.update_weights(
                            model=copy.deepcopy(model), global_round=round_, 
                                learning_rate = learning_rate, local_epochs = local_epochs, 
                                optimizer = optimizer)
            local_weights.append(copy.deepcopy(w))
            local_losses.append(copy.deepcopy(loss))

        # update global weights
        weights = average_weights(local_weights)
        model.load_state_dict(weights)

        loss_avg = sum(local_losses) / len(local_losses)
        train_loss.append(loss_avg)

        # Calculate avg training accuracy over all clients at every round
        list_acc, list_loss = [], []
        # the number of samples which are assigned to class y and belong to the sensitive group z
        n_yz = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
        loss_yz = {(0,0):0, (0,1):0, (1,0):0, (1,1):0}
        model.eval()
        for c in range(m):
            local_model = ClientUpdate(dataset=train_dataset,
                                        idxs=clients_idx[c], batch_size = batch_size, option = option, 
                                       penalty = penalty, lbd = lbd)
            # validation dataset inference
            acc, loss, n_yz_c, acc_loss, fair_loss, loss_yz_c = local_model.inference(model = model, 
                                                                                      option = option) 
            list_acc.append(acc)
            list_loss.append(loss)
            
            for yz in n_yz:
                n_yz[yz] += n_yz_c[yz]
                
                if option == "FairBatch": loss_yz[yz] += loss_yz_c[yz]
                
            print("Client %d: accuracy loss: %.2f | fairness loss %.2f | RD = %.2f = |%d/%d-%d/%d| " % (
                c, acc_loss, fair_loss, RD(n_yz_c), n_yz_c[(1,1)], n_yz_c[(1,1)] + n_yz_c[(0,1)], 
                n_yz_c[(1,0)], n_yz_c[(1,0)] + n_yz_c[(0,0)]))
            
        if option == "FairBatch": # update the lambda

            if abs(loss_yz[(0,0)]/m_yz[(0,0)] - loss_yz[(1,0)]/m_yz[(1,0)]) >= \
                abs(loss_yz[(0,1)]/m_yz[(0,1)] - loss_yz[(1,1)]/m_yz[(1,1)]):
                lbd[(0,0)] -= alpha * (2*int((loss_yz[(0,0)]/m_yz[(0,0)] - loss_yz[(1,0)]/m_yz[(1,0)]) > 0)-1)
                lbd[(0,1)] = (m_yz[(0,0)] + m_yz[(0,1)])/train_dataset.y.shape[0] - lbd[(0,0)]
            else:
                lbd[(1,0)] -= alpha * (2*int((loss_yz[(0,1)]/m_yz[(0,1)] - loss_yz[(1,1)]/m_yz[(1,1)]) > 0)-1)
                lbd[(1,1)] = (m_yz[(1,0)] + m_yz[(1,1)])/train_dataset.y.shape[0] - lbd[(1,0)]
            
        train_accuracy.append(sum(list_acc)/len(list_acc))

        # print global training loss after every 'i' rounds
        if (round_+1) % print_every == 0:
            print(f' \nAvg Training Stats after {round_+1} global rounds:')
            if option != "FairBatch":
                print("Training loss: %.2f | Validation accuracy: %.2f%% | Validation RD: %.2f" % (
                     np.mean(np.array(train_loss)), 
                    100*train_accuracy[-1], RD(n_yz)))
            else:
                print("Training loss: %.2f | Training accuracy: %.2f%% | Training RD: %.2f" % (
                     np.mean(np.array(train_loss)), 
                    100*train_accuracy[-1], RD(n_yz)))

    # Test inference after completion of training
    test_acc, test_loss, rd= test_inference(model, test_dataset, batch_size)

    print(f' \n Results after {num_rounds} global rounds of training:')
    print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
    print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

    # Compute RD: risk difference - fairness metric
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    print("|---- Test RD: {:.2f}".format(rd))

    print('\n Total Run Time: {0:0.4f} sec'.format(time.time()-start_time))

In [7]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), optimizer = 'sgd', learning_rate = 0.01)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 90.878746
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 53.193573
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 40.159161
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 45.825268
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 49.567120
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 47.455147
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 50.780552
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 40.212395
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 51.044689
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 36.907444
| Global Round : 0 | Local Epoch : 0 | [0/18116 (0%)]	Batch Loss: 93.284721
| Global Round : 0 | Local Epoch : 0 | [6400/18116 (35%)]	Batch Loss: 49.114399
| Global Round : 0 | Local Epoc

 20%|██        | 1/5 [00:03<00:15,  3.91s/it]

Client 0: accuracy loss: 103.63 | fairness loss 0.43 | RD = 0.30 = |101/592-669/1421| 
Client 1: accuracy loss: 60.87 | fairness loss 0.61 | RD = 0.25 = |56/421-313/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 48.40 | Validation accuracy: 80.87% | Validation RD: 0.28

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/18116 (0%)]	Batch Loss: 30.665058
| Global Round : 1 | Local Epoch : 0 | [6400/18116 (35%)]	Batch Loss: 41.343723
| Global Round : 1 | Local Epoch : 0 | [12800/18116 (70%)]	Batch Loss: 36.266209
| Global Round : 1 | Local Epoch : 1 | [0/18116 (0%)]	Batch Loss: 34.759308
| Global Round : 1 | Local Epoch : 1 | [6400/18116 (35%)]	Batch Loss: 40.490913
| Global Round : 1 | Local Epoch : 1 | [12800/18116 (70%)]	Batch Loss: 44.845722
| Global Round : 1 | Local Epoch : 2 | [0/18116 (0%)]	Batch Loss: 31.810749
| Global Round : 1 | Local Epoch : 2 | [6400/18116 (35%)]	Batch Loss: 47.987617
| Global Round : 1 | Local Epoch : 2 | [12800/181

 40%|████      | 2/5 [00:07<00:11,  3.86s/it]

...

 80%|████████  | 4/5 [00:15<00:03,  3.83s/it]

Client 0: accuracy loss: 100.26 | fairness loss 0.59 | RD = 0.28 = |82/592-594/1421| 
Client 1: accuracy loss: 59.69 | fairness loss 0.70 | RD = 0.27 = |48/421-315/823| 
 
Avg Training Stats after 4 global rounds:
Training loss: 45.82 | Validation accuracy: 82.61% | Validation RD: 0.28

 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 49.766724
| Global Round : 4 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 47.591393
| Global Round : 4 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 47.119442
| Global Round : 4 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 41.633862
| Global Round : 4 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 58.509113
| Global Round : 4 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 34.114101
| Global Round : 4 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 53.478104
| Global Round : 4 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 51.273594
| Global Round : 4 | Local Epoch : 4 | [0/11188 (0%)]	Ba

100%|██████████| 5/5 [00:18<00:00,  3.79s/it]

Client 0: accuracy loss: 98.34 | fairness loss 0.65 | RD = 0.19 = |77/592-455/1421| 
Client 1: accuracy loss: 58.96 | fairness loss 0.76 | RD = 0.19 = |42/421-240/823| 
 
Avg Training Stats after 5 global rounds:
Training loss: 45.48 | Validation accuracy: 83.66% | Validation RD: 0.19


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 83.66%
|---- Test Accuracy: 84.58%
|---- Test RD: 0.16

 Total Run Time: 19.2994 sec


In [6]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), 
      "Zafar", penalty = 50, optimizer = 'sgd', learning_rate = 0.01,
     num_rounds = 5, local_epochs = 10)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 91.087540
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 107.725006
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 45.530258
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 57.458233
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 61.342957
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 63.603767
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 67.874146
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 100.446243
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 54.705631
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 102.820938
| Global Round : 0 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 44.603512
| Global Round : 0 | Local Epoch : 5 | [6400/11188 (57%)]	Batch Loss: 64.342941
| Global Round : 0 | Local E

 20%|██        | 1/5 [00:07<00:30,  7.62s/it]

Client 0: accuracy loss: 107.03 | fairness loss 0.60 | RD = 0.30 = |98/592-661/1421| 
Client 1: accuracy loss: 63.55 | fairness loss 0.87 | RD = 0.24 = |57/421-308/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 81.47 | Validation accuracy: 81.13% | Validation RD: 0.28

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/18116 (0%)]	Batch Loss: 47.653412
| Global Round : 1 | Local Epoch : 0 | [6400/18116 (35%)]	Batch Loss: 33.277695
| Global Round : 1 | Local Epoch : 0 | [12800/18116 (70%)]	Batch Loss: 43.634350
| Global Round : 1 | Local Epoch : 1 | [0/18116 (0%)]	Batch Loss: 46.054932
| Global Round : 1 | Local Epoch : 1 | [6400/18116 (35%)]	Batch Loss: 179.703430
| Global Round : 1 | Local Epoch : 1 | [12800/18116 (70%)]	Batch Loss: 60.887173
| Global Round : 1 | Local Epoch : 2 | [0/18116 (0%)]	Batch Loss: 61.975349
| Global Round : 1 | Local Epoch : 2 | [6400/18116 (35%)]	Batch Loss: 87.575302
| Global Round : 1 | Local Epoch : 2 | [12800/181

 40%|████      | 2/5 [00:15<00:22,  7.66s/it]

...

 80%|████████  | 4/5 [00:30<00:07,  7.58s/it]

Client 1: accuracy loss: 63.73 | fairness loss 0.26 | RD = 0.21 = |46/421-265/823| 
 
Avg Training Stats after 4 global rounds:
Training loss: 72.68 | Validation accuracy: 83.84% | Validation RD: 0.22

 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 56.079002
| Global Round : 4 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 32.749493
| Global Round : 4 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 44.470371
| Global Round : 4 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 57.926239
| Global Round : 4 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 35.525433
| Global Round : 4 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 108.887573
| Global Round : 4 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 47.313347
| Global Round : 4 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 67.285103
| Global Round : 4 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 49.684402
| Global Round : 4 | Local Epoch : 4 | [6400/11188 (57%)]	Batch L

100%|██████████| 5/5 [00:37<00:00,  7.57s/it]

Client 0: accuracy loss: 103.54 | fairness loss 0.87 | RD = 0.26 = |66/592-528/1421| 
Client 1: accuracy loss: 62.40 | fairness loss 1.02 | RD = 0.25 = |40/421-280/823| 
 
Avg Training Stats after 5 global rounds:
Training loss: 71.61 | Validation accuracy: 83.51% | Validation RD: 0.26


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 83.51%
|---- Test Accuracy: 84.46%
|---- Test RD: 0.20

 Total Run Time: 38.2369 sec


In [8]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), 
      option = "FairBatch", optimizer = 'sgd', learning_rate = 0.01, 
      num_rounds = 20, local_epochs = 10, alpha = 0.005)

  0%|          | 0/20 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 88.324371
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 76.816750
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 54.973553
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 53.144211
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 45.883881
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 57.148705
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 45.879372
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 50.717587
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 51.708126
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 43.595169
| Global Round : 0 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 42.206825
| Global Round : 0 | Local Epoch : 5 | [6400/11188 (57%)]	Batch Loss: 62.472843
| Global Round : 0 | Local Epoc

  5%|▌         | 1/20 [00:08<02:44,  8.67s/it]

| Global Round : 0 | Local Epoch : 9 | [17550/18116 (106%)]	Batch Loss: 44.065224
Client 0: accuracy loss: 104.66 | fairness loss 0.42 | RD = 0.40 = |91/592-780/1421| 
Client 1: accuracy loss: 61.11 | fairness loss 0.59 | RD = 0.32 = |52/421-362/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 50.86 | Training accuracy: 79.28% | Training RD: 0.37

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 48.539951
| Global Round : 1 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 51.615456
| Global Round : 1 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 44.478088
| Global Round : 1 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 53.917126
| Global Round : 1 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 43.242596
| Global Round : 1 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 41.952991
| Global Round : 1 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 45.442455
| Global Round : 1 | Local Epoch : 3 | [6400/11188 (57%)]	

 10%|█         | 2/20 [00:17<02:36,  8.71s/it]

...

 95%|█████████▌| 19/20 [02:50<00:09,  9.26s/it]

Client 0: accuracy loss: 96.47 | fairness loss 1.44 | RD = 0.38 = |14/592-575/1421| 
Client 1: accuracy loss: 57.35 | fairness loss 1.59 | RD = 0.36 = |10/421-317/823| 
 
Avg Training Stats after 19 global rounds:
Training loss: 47.23 | Training accuracy: 82.49% | Training RD: 0.37

 | Global Training Round : 20 |

| Global Round : 19 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 260.192902
| Global Round : 19 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 39.220600
| Global Round : 19 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 221.713791
| Global Round : 19 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 33.998917
| Global Round : 19 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 266.978119
| Global Round : 19 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 40.089378
| Global Round : 19 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 289.233948
| Global Round : 19 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 42.057045
| Global Round : 19 | Local Epoch : 4 | [0/1118

100%|██████████| 20/20 [03:00<00:00,  9.01s/it]

Client 0: accuracy loss: 95.98 | fairness loss 1.49 | RD = 0.36 = |13/592-536/1421| 
Client 1: accuracy loss: 57.10 | fairness loss 1.64 | RD = 0.33 = |10/421-288/823| 
 
Avg Training Stats after 20 global rounds:
Training loss: 47.20 | Training accuracy: 83.05% | Training RD: 0.34


 
 Results after 20 global rounds of training:
|---- Avg Train Accuracy: 83.05%
|---- Test Accuracy: 84.31%
|---- Test RD: 0.27

 Total Run Time: 180.6578 sec
